In [1]:
from langchain_community.document_loaders import JSONLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts import PromptTemplate,ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.agents import Tool, initialize_agent, create_openai_functions_agent, AgentExecutor
from langchain.agents.agent import RunnableAgent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain_chroma import Chroma
from pydantic import BaseModel
from dotenv import load_dotenv
from typing import Optional
import pandas as pd
import os
import json
import gradio as gr
from gtts import gTTS
import tempfile
import base64
import numpy as np
import requests
import pandas as pd
from tqdm import tqdm
import csv
from datetime import datetime


In [4]:
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ORS_API_KEY = os.getenv("ORS_API_KEY")

# Check the key

if not OPENAI_API_KEY:
    print("No OpenAI API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not OPENAI_API_KEY.startswith("sk-proj-"):
    print("An OpenAI API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif OPENAI_API_KEY.strip() != OPENAI_API_KEY:
    print("An OpenAI API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("OpenAi API key found and looks good so far!")
    
if not ORS_API_KEY:
    print("No ORS API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif ORS_API_KEY.strip() != ORS_API_KEY:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("ORS API key found and looks good so far!")

OpenAi API key found and looks good so far!
ORS API key found and looks good so far!


In [67]:
# === SETUP ===
DATASET_PATH = "AutoRAG_Dataset"
manual_path = os.path.join(DATASET_PATH, "car_manuals", "manuals.json")
weather_path = os.path.join(DATASET_PATH, "weather_data", "weather_data.csv")
poi_path = os.path.join(DATASET_PATH, "poi_data", "poi_data.csv")
PROFILE_PATH = os.path.join(DATASET_PATH, "driver_profiles", "profiles.json")
TELEMETRY_PATH = os.path.join(DATASET_PATH, "telemetry", "telemetry_logs.csv")
VOICE_LOG_PATH = os.path.join(DATASET_PATH, "voice_queries", "voice_queries.csv")
CALENDAR_PATH = os.path.join(DATASET_PATH, "calendar_events", "calendar_events.csv")

In [8]:
# === VECTOR INDEX BUILDING FROM MULTIPLE SOURCES ===
def build_vectorstore_from_multiple_sources(index_path="car_manual_index"):
    documents = []

    
    # Car Manuals
    with open(manual_path) as f:
        manuals = json.load(f)
    for m in manuals:
        documents.append(Document(
            page_content=m["content"],
            metadata={"type": "manual", "doc_id": m["doc_id"]}
        ))


    # QA Triplets
    with open("AutoRAG_Dataset/qa_triplets/qa_pairs.json") as f:
        qas = json.load(f)
    for qa in qas:
        content = f"Q: {qa['question']}\nContext: {qa['retrieved_context']}\nA: {qa['answer']}"
        documents.append(Document(
            page_content=content,
            metadata={"type": "qa_triplet", "qa_id": qa["qa_id"]}
        ))


    # POI
    poi_df = load_poi_data()
    for _, row in poi_df.iterrows():
        content = f"{row['type']} - {row['name']} in {row['city']}, Rating: {row['rating']}, Distance: {row['distance_km']} km"
        documents.append(Document(
            page_content=content,
            metadata={"type": "poi", "city": row["city"], "poi_type": row["type"]}
        ))

    # === Helper ===
    def format_saved_locations(locs):
        return "; ".join(locs)
    
    # === 1. Driver Profiles ===
    with open(PROFILE_PATH) as f:
        profiles = json.load(f)
    
    profile_lookup = {p["user_id"]: p for p in profiles}
    
    for profile in profiles:
        content = (
            f"This is information about user {profile['name']} (user ID: {profile['user_id']}). "
            f"They are {profile['age']} years old and prefer {profile['preferred_music']} music. "
            f"Their home address is {profile['home_address'].replace(chr(10), ', ')}, and they work at {profile['work_address'].replace(chr(10), ', ')}. "
            f"Their driving style is {profile['driving_style']}. "
            f"Saved locations include: {format_saved_locations(profile['saved_locations'])}."
        )
        documents.append(Document(page_content=content, metadata={"type": "profile", "user_id": profile['user_id']}))
    
    # === 2. Telemetry Logs ===
    telemetry_df = pd.read_csv(TELEMETRY_PATH)
    for user_id, user_df in tqdm(telemetry_df.groupby("user_id"), desc="Processing telemetry"):
        profile = profile_lookup.get(user_id, {})
        user_name = profile.get("name", user_id)
        summary = (
            f"Telemetry summary for user {user_name} (user ID: {user_id}). "
            f"{len(user_df)} data points recorded across trips. "
            f"Speed ranged from {user_df['speed_kmph'].min()} to {user_df['speed_kmph'].max()} km/h. "
            f"Battery ranged from {user_df['battery_level_percent'].min()}% to {user_df['battery_level_percent'].max()}%. "
            f"Tire pressure was typically around {user_df['tire_pressure_psi'].mean():.2f} PSI. "
            f"Cabin temperature averaged {user_df['cabin_temp_c'].mean():.2f}°C."
        )
        documents.append(Document(page_content=summary, metadata={"type": "telemetry", "user_id": user_id}))

    # === 3. Voice Queries ===
    queries_df = pd.read_csv(VOICE_LOG_PATH)
    for user_id, user_df in tqdm(queries_df.groupby("user_id"), desc="Processing voice queries"):
        profile = profile_lookup.get(user_id, {})
        user_name = profile.get("name", user_id)
        content = (
            f"Voice interaction history for user {user_name} (user ID: {user_id}): "
            f"They asked the assistant the following types of queries: "
            f"{'; '.join(user_df['query_text'].unique()[:10])}."
        )
        documents.append(Document(page_content=content, metadata={"type": "voice_queries", "user_id": user_id}))
    
    # === 4. Calendar Events ===
    calendar_df = pd.read_csv(CALENDAR_PATH)
    for user_id, user_df in tqdm(calendar_df.groupby("user_id"), desc="Processing calendar"):
        profile = profile_lookup.get(user_id, {})
        user_name = profile.get("name", user_id)
        events = ", ".join(user_df["title"].unique())
        content = (
            f"Calendar for user {user_name} (user ID: {user_id}): "
            f"Scheduled events include: {events}. "
            f"Typical durations are around {user_df['duration_minutes'].mean():.1f} minutes."
        )
        documents.append(Document(page_content=content, metadata={"type": "calendar", "user_id": user_id}))


    # Embedding
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    split_docs = splitter.split_documents(documents)
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(split_docs, embeddings)
    db.save_local(index_path)
    return db



In [10]:
# === DATA LOADING FUNCTIONS ===
def load_weather_data():
    return pd.read_csv(weather_path)

def load_poi_data():
    return pd.read_csv(poi_path)


In [12]:
# === Weather Tool ===
class WeatherInput(BaseModel):
    city: str

@tool("get_weather", args_schema=WeatherInput)
def get_weather_tool(city: str) -> str:
    """Get current weather info for a city using Open-Meteo API."""
    url = f"https://api.open-meteo.com/v1/forecast?latitude=40.71&longitude=-74.01&current_weather=true"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        weather = data['current_weather']
        return f"Current weather in {city}: {weather['temperature']}°C, Wind {weather['windspeed']} km/h."
    return "Sorry, unable to fetch weather."


# === Navigation Tool ===
class NavigationInput(BaseModel):
    from_location: str
    to_location: str
    

@tool("get_directions", args_schema=NavigationInput)
def get_directions_tool(from_location: str, to_location: str) -> str:
    """Get driving directions from one location to another using OpenRouteService."""
    def geocode(location: str) -> list:
        geo_url = f"https://api.openrouteservice.org/geocode/search"
        headers = {
            "Authorization": ORS_API_KEY
        }
        params = {
            "api_key": ORS_API_KEY,
            "text": location,
            "size": 1
        }
        resp = requests.get(geo_url, headers=headers, params=params)
        try:
            return resp.json()['features'][0]['geometry']['coordinates']
        except Exception as e:
            raise ValueError(f"Could not geocode '{location}': {resp.text}")

    headers = {
        "Authorization": ORS_API_KEY,
        "Content-Type": "application/json"
    }

    try:
        from_coords = geocode(from_location)
        to_coords = geocode(to_location)
    except ValueError as e:
        return str(e)

    body = {
        "coordinates": [from_coords, to_coords]
    }

    url = "https://api.openrouteservice.org/v2/directions/driving-car"
    response = requests.post(url, headers=headers, json=body)

    try:
        data = response.json()
    except:
        return f"Invalid JSON returned: {response.text}"

    if "features" not in data:
        return f"Unexpected response format:\n{data}"

    try:
        steps = data['features'][0]['properties']['segments'][0]['steps']
        directions = "\n".join([f"{i+1}. {s['instruction']}" for i, s in enumerate(steps)])
        return "Navigation Steps:\n" + directions
    except Exception as e:
        return f"Failed to parse directions: {e}\nFull response:\n{data}"


# === User Profile Tool ===
class UserProfileInput(BaseModel):
    name: str

@tool("get_user_profile", args_schema=UserProfileInput)
def get_user_profile_tool(name: str) -> str:
    """Retrieve driver profile based on name."""
    with open(PROFILE_PATH) as f:
        profiles = json.load(f)

    name = name.strip().lower()

    for p in profiles:
        if p['name'].split()[0].lower() == name:
            return (
                f"User {p['name']}: {p['age']} yrs\n"
                f"Home: {p['home_address']}\n"
                f"Work: {p['work_address']}\n"
                f"Driving Style: {p['driving_style']}\n"
                f"Preferred Music: {p['preferred_music']}\n"
                f"Saved Locations:\n- " + "\n- ".join(p['saved_locations'])
            )
    
    return "User not found."


# === RAG Retrieval Tool ===
class RAGInput(BaseModel):
    query: str

@tool("smart_rag", args_schema=RAGInput)
def smart_rag_tool(query: str) -> str:
    """Answer queries using in-vehicle data including manuals, FAQs, and Points Of Interests, trip logs,Telemetry Logs,Voice Query History,Calander Events  ."""
    vectorstore = FAISS.load_local("car_manual_index", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever()
    docs = retriever.get_relevant_documents(query)
    return "\n".join([doc.page_content for doc in docs]) if docs else "No relevant info found in the manual."




class LogFullInteractionInput(BaseModel):
    name: str         # Full name input
    query: str        # User's spoken/typed input
    response: str     # LLM model's response

@tool("log_user_interaction", args_schema=LogFullInteractionInput)
def log_user_interaction_tool(name: str, query: str, response: str) -> str:
    """Logs a user query and the corresponding model response."""
    # Load existing profiles to resolve name to user_id
    with open(PROFILE_PATH) as f:
        profiles = json.load(f)

    name = name.strip().lower()
    matched_user = next((p for p in profiles if name in p["name"].lower()), None)
    if not matched_user:
        return f"⚠️ No user found matching name '{name}'"

    user_id = matched_user["user_id"]
    full_name = matched_user["name"]
    timestamp = datetime.utcnow().isoformat()
    intent = query.strip().split()[0].lower() if query.strip() else "unknown"

    log_entry = {
        "user_id": user_id,
        "name": full_name,
        "timestamp": timestamp,
        "query_text": query,
        "response_text": response,
        "intent": intent
    }

    # Ensure file has header if it doesn't exist
    file_exists = os.path.exists(VOICE_LOG_PATH)
    with open(VOICE_LOG_PATH, "a", newline='') as f:
        writer = csv.DictWriter(
            f,
            fieldnames=["user_id", "name", "timestamp", "query_text", "response_text", "intent"]
        )
        if not file_exists:
            writer.writeheader()
        writer.writerow(log_entry)

    return f"✅ Logged interaction for {full_name} ({user_id})"

In [14]:
def get_agent_executor():
    tools = [
        get_weather_tool,
        get_directions_tool,
        get_user_profile_tool,
        smart_rag_tool,
        log_user_interaction_tool
    ]

    llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
            """
            You are AutoRAG++, an intelligent in-vehicle assistant for drivers.

            You have access to various tools and a rich internal knowledge base powered by vector search. 
            The knowledge base contains multiple types of documents and data that you can retrieve and use to answer queries accurately and helpfully.
            
            Here’s what your memory includes:
            
            📘 **Car Manuals**: Detailed information about car parts, maintenance instructions, settings, and troubleshooting steps.
            
            💬 **QA Triplets**: Frequently asked questions from users with context and pre-generated answers.
            
            📍 **Points of Interest (POIs)**: Places like restaurants, hospitals, and gas stations along with their type, location, rating, and distance.
            
            👤 **Driver Profiles**: Data about each user including name, age, home and work addresses, driving style, preferred music, and saved locations.
            
            📊 **Telemetry Logs**: Trip-level data including speed, battery level, tire pressure, and cabin temperature over time for each user.
            
            🗣️ **Voice Query History**: Past voice/text interactions made by users with associated timestamps and inferred intent.
            
            📅 **Calendar Events**: Upcoming user events such as meetings or service appointments with location and time.
            
    
            
            🛠️ You also have access to tools that allow you to:
            - Navigate between places using real-time routing
            - Retrieve current weather
            - Look up specific user profiles
            - Log new user interactions (query + response)
            - Retrieve relevant information using RAG (retrieval-augmented generation)
            
            🎯 Your job is to use this knowledge and the tools to help the driver:
            - Answer questions about car functionality or past trips
            - Find POIs or weather in any location
            - Understand driving stats or calendar reminders
            - Act as a friendly voice assistant that logs every interaction
            - Keep responses short, clear, and spoken in a natural way

            You should log every user query and your response using the `log_user_interaction` tool. Provide the user's name, query, and your response.
            Always try to personalize your responses using user-specific data and keep answers natural for speech (as your response will be read aloud).
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

    prompt = prompt.partial()

    agent: RunnableAgent = create_openai_functions_agent(
        llm=llm,
        prompt=prompt,
        tools=tools
    )

    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    executor = AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True
    )

    return executor

In [24]:
build_vectorstore_from_multiple_sources(index_path="car_manual_index")
agent = get_agent_executor()

Processing calendar: 100%|███████████████████| 50/50 [00:00<00:00, 11881.88it/s]


In [36]:
response = agent.invoke(
    {"input": "How is the weather in atlanta today?"}
)
print(response["output"])



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': 'Atlanta'}`


Current weather in Atlanta: 21.8°C, Wind 9.4 km/h.The weather in Atlanta today is about 22 degrees Celsius with a light wind of 9 kilometers per hour. Would you like to know anything else?

> Finished chain.
The weather in Atlanta today is about 22 degrees Celsius with a light wind of 9 kilometers per hour. Would you like to know anything else?


In [60]:
# === UI ===
def launch_ui(chain):
    def respond(audio_file, text):
        if text:
            query = text
        elif audio_file:
            import speech_recognition as sr
            recognizer = sr.Recognizer()
            with sr.AudioFile(audio_file) as source:
                audio = recognizer.record(source)
            try:
                query = recognizer.recognize_google(audio)
            except sr.UnknownValueError:
                return gr.update(value=""), "Sorry, I couldn't understand the audio.", ""
            except sr.RequestError:
                return gr.update(value=""), "Speech recognition service error.", ""
        else:
            return gr.update(value=""), "Please provide input.", ""

        result = chain.invoke({"input": query})["output"]
        

        tts = gTTS(result)
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
            temp_audio_path = f.name
            tts.save(temp_audio_path)

        with open(temp_audio_path, "rb") as f:
            audio_data = f.read()
        audio_base64 = base64.b64encode(audio_data).decode("utf-8")

        audio_html = f"""
        <audio id='response-audio' controls autoplay style='width:100%;'>
            <source src='data:audio/mpeg;base64,{audio_base64}' type='audio/mpeg'>
            Your browser does not support the audio element.
        </audio>
        <script>
            const audio = document.getElementById("response-audio");
            if (audio) {{
                audio.playbackRate = 1.5;
                audio.play().catch(() => {{ console.warn("Autoplay blocked."); }});
            }}
        </script>
        """
        return gr.update(value=""), result, audio_html
    
    gr.Interface(
        fn=respond,
        inputs=[
            gr.Audio(type="filepath", label="🎙️ Speak your query (or type below)"),
            gr.Textbox(label="✏️ Or type your query")
        ],
        outputs=[
            gr.Textbox(label="✏️", interactive=True),
            gr.Textbox(label="🧠 Response", interactive=False),
            gr.HTML(label="🔊 Audio Reply")
        ],
        
        title="🚗 NaviZen — A calm, reliable In-Vehicle voice co-pilot that understands you.",
        theme="soft"
    ).launch()


In [65]:
# === MAIN RUN ===
if __name__ == "__main__":
    print("🚀 Building index and launching assistant...")
    #build_vectorstore_from_multiple_sources(index_path="car_manual_index")
    weather_df = load_weather_data()
    poi_df = load_poi_data()
    rag_chain = get_agent_executor()
    launch_ui(rag_chain)

🚀 Building index and launching assistant...
* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [81]:
!pip install gTTS

In [65]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.7 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip install langchain_community

In [61]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.7 MB/s eta 0:00:0000:0100:01m


In [12]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.3/426.3 kB 1.8 MB/s eta 0:00:0000:0100:01m


In [ ]:
!!pip install jq